In [1]:
# Imports
import scipy.stats as stats
import numpy as np
import pandas as pd
from pandasai import PandasAI
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import explore as e
import prepare
import warnings
warnings.filterwarnings("ignore")
import plotly.express

In [2]:
df = prepare.prepare_edu()

In [3]:
train, validate, test = prepare.split(df)

In [4]:
X_train, y_train, X_validate, y_validate, X_test, y_test= e.modeling_split(train,validate,test)

In [6]:
X_test = X_test.replace({False : 0,
              True : 1 }, inplace = True)

X_validate = X_validate.replace({False : 0,
              True : 1 }, inplace = True)

X_train = X_train.replace({False : 0,
              True : 1 }, inplace = True)


In [7]:
validate

,is_male,free_reduced_lunch,test_prep_completed,practicesport,is_first_child,nrsiblings,rides_bus,math_score,reading_score,writing_score,...,parent_educ_high_school,parent_educ_masters_degree,parent_educ_some_college,parent_educ_some_high_school,parent_marital_status_married,parent_marital_status_single,parent_marital_status_widowed,wkly_study_hours_<_5,wkly_study_hours_>_10,risk_rating
29168,1,0,1,1,0,5.0,1,78,83,80,...,False,False,True,False,False,False,False,True,False,high_risk
20039,1,1,0,0,1,1.0,0,63,67,59,...,False,False,True,False,True,False,False,False,False,at_risk
30511,1,0,0,1,0,2.0,1,84,69,67,...,False,False,True,False,True,False,False,False,False,at_risk
6163,1,1,1,1,0,3.0,1,58,62,53,...,True,False,False,False,False,False,False,False,True,low_risk
30604,1,0,0,1,0,1.0,1,93,77,79,...,False,False,False,True,True,False,False,False,False,high_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20659,1,0,0,1,1,3.0,1,75,77,82,...,False,False,False,False,False,True,False,False,True,high_risk
1419,1,1,0,1,1,2.0,1,60,64,52,...,True,False,False,False,True,False,False,True,False,at_risk
20234,0,0,0,1,1,4.0,1,84,96,93,...,False,False,True,False,True,False,False,True,False,high_risk
5221,1,0,1,1,1,0.0,0,90,84,88,...,False,False,False,False,True,False,False,False,False,high_risk
